In [1]:
import json
import os

In [1]:
def ParsingJson(textname, filename):
    with open("./data/raw/sample/"+textname+"/"+filename+".json", "r", encoding="utf8") as f:
        contents = f.read()
        file = json.loads(contents)
    keyJson = []
    if textname=="chat":
        keyJson = "utterances"
    elif textname=="news":
        keyJson = "sentences"
    
    countSentences = len(file[keyJson])
    
    result_list = []
    
    for k in range(countSentences):
        sentence = file[keyJson][k]["text"]

        len_timex3 = len(file[keyJson][k]["timex3"])
        len_event = len(file[keyJson][k]["event"])
        info_list = [[] for i in range(len_event+len_timex3)]

        for i in range(len_event):
            event_s, event_e = file[keyJson][k]["event"][i]["extent"]
            info_list[i].append(event_s)
            info_list[i].append(event_e)
            info_list[i].append(file[keyJson][k]["event"][i]["class"])
            info_list[i].append(file[keyJson][k]["event"][i]["id"])
            info_list[i].append(file[keyJson][k]["event"][i]["text"])

        for i in range(len_timex3):
            timex3_s, timex3_e = file[keyJson][k]["timex3"][i]["extent"]
            info_list[i+len_event].append(timex3_s)
            info_list[i+len_event].append(timex3_e)
            info_list[i+len_event].append(file[keyJson][k]["timex3"][i]["type"])
            info_list[i+len_event].append(file[keyJson][k]["timex3"][i]["id"])
            info_list[i+len_event].append(file[keyJson][k]["timex3"][i]["text"])
            

        info_list.sort()
        print(info_list)

        named_entity = [0 for i in range(len(sentence))]
        for i in range(len(info_list)):
            start = info_list[i][0]
            end = info_list[i][1]
            netype = info_list[i][2]
            BorI = True
            for j in range(start, end):
                if BorI:
                    named_entity[j] = (netype, '-B')
                    BorI = False
                else:
                    named_entity[j] = (netype, '-I')

        label = []
        for i in range(len(sentence)):
            if named_entity[i]:
                label.append(named_entity[i][0]+named_entity[i][1])
            else:
                label.append('O')
            
        result = sentence + '\t'
        for i in range(len(label)):
            result += label[i] + ' '
        result = result.strip()
        result_list.append(result)
        
        # 탭 문자 하나 더 넣고 tlink 넣기
        for i in range(len(result_list)):
            result_list[i] = result_list[i] + '\t'
            
        tlinkList = file["tlink"]
        for i in range(len(tlinkList)):
            timex3NE = tlinkList[i]["arg1"]
            eventNE = tlinkList[i]["arg2"]
            index = int(timex3NE.split('_')[1][1:])
            if (infolist[index][3] == timex3NE):
                if (infolist[index][3])] == eventNE):
                    time_link = infolist[timex3NE.split('_')[2]][4]
                    event_link = infolist[eventNE.split('_')[2]][4]
                    tlink_type = tlinkList[i]["relType"]
                    res_tlink = time_link+" "+event_link+" "+tlink_type
                    result_list[index] = result_list[index] + res_tlink
        # 예상 result_list[i]의 모습
        # 문장 \t BIO tag \t 시간표현 \s 사건표현 \s 관계
    return result_list

In [3]:
def saveTxt(textname, filename, res):
    f = open("./data/biotag/"+textname+"/"+filename, "w")
    for i in range(len(res)):
        f.write(res[i]+'\n')
    f.close()

In [4]:
# 파일이름 받아오기

text_name = ["chat", "news"]
file_list = []
for i in range(len(text_name)):
    file_list.append(os.listdir("./data/raw/sample/"+text_name[i]))